In [9]:
%%capture
!pip install -U optuna pytorch_lightning==1.8.5.post0 rouge-score transformers sentencepiece pandas

In [3]:
df_syn = pd.read_parquet("/home/gvillarroel/dev/synthetic-data-for-text/datasets/economicos/synth/data/tddpm_mlp_noise_27574.parquet")
df_syn

,url,_price,property_type,transaction_type,state,county,rooms,bathrooms,m_built,m_size,source
0,0,4881.057824,Departamento,Venta,Metropolitana de Santiago,Recoleta,2.0,2.0,4838.0,95.0,El Mercurio
1,1,90.642593,Departamento,Arriendo,Metropolitana de Santiago,Vitacura,2.0,4.0,140.0,235.0,El Mercurio
2,2,14000.000000,Casa,Venta,Araucanía,Temuco,4.0,3.0,325.0,6075.0,El Mercurio
3,3,24.333909,Departamento,Arriendo,Metropolitana de Santiago,Providencia,3.0,2.0,90.0,90.0,El Mercurio
4,4,24.293454,Departamento,Arriendo,Metropolitana de Santiago,Las Condes,2.0,2.0,95.0,84.0,El Mercurio
...,...,...,...,...,...,...,...,...,...,...,...
27569,27569,24999.061051,Departamento,Venta,Metropolitana de Santiago,Las Condes,3.0,4.0,134.0,25.0,El Mercurio
27570,27570,4594.259027,Departamento,Venta,Metropolitana de Santiago,Las Condes,1.0,1.0,53.0,49.0,El Mercurio
27571,27571,15.558321,Departamento,Arriendo,Coquimbo,La Serena,2.0,2.0,60.0,75.0,El Mercurio
27572,27572,8.997128,Departamento,Arriendo,Metropolitana de Santiago,Santiago,1.0,1.0,35.0,35.0,El Mercurio


In [2]:
import pandas as pd
import numpy as np
#.replace(to_replace="-1", value=np.nan)
df = pd.read_parquet('../datasets/economicos/synth/split/train.parquet').replace(to_replace="None", value=np.nan).replace(to_replace=-1, value=np.nan)
display(df.shape)

CHAR_SEP = " "
df.sample(3)


(22059, 17)

,url,description,price,property_type,transaction_type,state,county,publication_date,rooms,bathrooms,m_built,m_size,source,title,address,owner,_price
436070,https://www.economicos.cl/propiedades/oficina-...,MODERNO EDIFICIO EN EXCLUSIVO SECTOR DE CUMBRE...,15.000 UF,Oficina o Casa Oficina,Venta,Biobío,San Pedro De La Paz,2022-07-21 00:00:13,8.0,4.0,171.0,450.0,El Mercurio,Oficina o Casa Oficina en Venta en San Pedro D...,"Avenida Andalue, Cumbres de Andalue San Pedro ...",Agente 365,15000.0
529723,https://www.economicos.cl/propiedades/casa-en-...,"Casona histórica de 3 pisos, totalmente resta...",20.000 UF,Casa,Venta,Valparaíso,Valparaíso,2022-02-23 00:00:08,4.0,2.0,500.0,500.0,El Mercurio,Casa en Venta en Valparaíso 4 dormitorios 2 baños,"CERRO ALEGRE Valparaíso, Valparaíso",Proa Gestión Inmobiliaria,20000.0
420833,https://www.economicos.cl/propiedades/departam...,Espectacular departamento en Costa Cachagua de...,36.100 UF,Departamento,Venta,Valparaíso,Puchuncaví,2022-07-21 00:00:26,5.0,5.0,320.0,390.0,El Mercurio,Departamento en Venta en Puchuncaví 5 dormitor...,"Vista Panorámica / Costa Cachagua Puchuncaví,...",Bórquez y Asociados Limitada,36100.0


In [11]:
def convert(row, column="description"):
    return {
        "text": [f"{column} this data:", f"""
fecha {row.publication_date.strftime('%Y-%m-%d')}
precio {row.price}
tipo {row.property_type}
transacción {row.transaction_type}
región {row.state}
comuna {row.county}
dormitorios {row.rooms}
baños {row.rooms}
constuidos {row.m_built}
terreno {row.m_size}
precio_real {row._price}
titulo: {row.title}
dirección: {row.address}""".replace("\n", " ")],
        "target": row[column]
        }

print(
    df.sample(1).apply(convert, axis=1).iloc[-1]
)

{'text': ['description this data:', ' fecha 2022-07-21 precio 10.800 UF tipo Casa transacción Venta región Valparaíso comuna Viña del Mar dormitorios 5.0 baños 5.0 constuidos 216.0 terreno 332.0 precio_real 10800.0 titulo: Casa en Venta en Viña del Mar 5 dormitorios 4 baños dirección: Miraflores Viña del Mar, Valparaíso'], 'target': 'Linda casa de 2 pisos con amplias habitaciones  y gran estacionamiento para 4 autos.  Primer piso.:  Living con Chimenea  Comedor aparte con salida a jardin  Cocina con amplia Despensa  Desayunador  Pieza y baño de servicio  Sala de estar y 1 dormitorio   Baño completo.  Estacionamiento 4 autos  Lavadero exterior cerrado  Bodega   Segundo piso :  3 dormitorios  1 en suite amplio.  Baño completo  Ventanas termopanel. - Código Propiedad: JX8162'}


In [36]:
df_syn.apply(convert, axis=1)

AttributeError: 'Series' object has no attribute 'publication_date'

In [12]:
from functools import  partial
df_text = pd.concat([pd.DataFrame(df.apply(convert, axis=1).to_list()), pd.DataFrame(df.apply(partial(convert, column="title"), axis=1).to_list()), pd.DataFrame(df.apply(partial(convert, column="address"), axis=1).to_list())])
df_text.sample(3)

,text,target
6631,"[address this data:, fecha 2022-07-21 precio ...","CLARO SOLAR ESQUINA CAUPOLICAN Temuco, Araucanía"
1589,"[address this data:, fecha 2019-05-18 precio ...","Providencia, Metropolitana de Santiago"
12359,"[description this data:, fecha 2020-04-03 pre...","UF 21.500,00 Melipilla Hermosa parcela en cond..."


In [13]:
import argparse
from argparse import ArgumentParser
from os.path import join, isfile
from os import listdir
import optuna
from optuna.integration import PyTorchLightningPruningCallback
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from rouge_score import rouge_scorer
import shutil
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import  DataLoader, RandomSampler, SequentialSampler #Dataset,
from transformers import get_linear_schedule_with_warmup, AdamW
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [14]:
class MetricsCallback(pl.Callback):
    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [15]:
CHAR_SEP = " "
MAX_SRC_LEN = 150
MAX_TGT_LEN = 720
from functools import partial

class T5Finetuner(pl.LightningModule):

    def __init__(self, args, df, batch_size=8):
        super().__init__()
        self.save_hyperparameters()
        self.args = args
        self.model = T5ForConditionalGeneration.from_pretrained(self.args.model)
        self.tokenizer = T5Tokenizer.from_pretrained(self.args.model)
        self.data = df
        #self.scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.batch_size = batch_size
        self.decode = partial(self.tokenizer.decode, skip_special_tokens=True)

    def _encode_text(self, text_input, target):
      ctext = str(text_input)
      ctext = CHAR_SEP.join(ctext.split())
      target = str(target) #summarized text
      target = CHAR_SEP.join(target.split())
      source = self.tokenizer.batch_encode_plus([ctext], 
                                                max_length= MAX_SRC_LEN, 
                                                truncation=True,
                                                padding='max_length',
                                                return_tensors='pt')
      target = self.tokenizer.batch_encode_plus([target], 
                                                max_length=MAX_TGT_LEN,
                                                truncation=True,
                                                padding='max_length',
                                                return_tensors='pt')
      y = target['input_ids']
      target_id = y[:, :-1].contiguous()
      target_label = y[:, 1:].clone().detach()
      target_label[y[:, 1:] == self.tokenizer.pad_token_id] = -100 #in case the labels are not provided, empty string
      return source['input_ids'], source['attention_mask'], target_id, target_label
    
    def encode_text(self, text, target):
        source = self.tokenizer.batch_encode_plus([text], 
                                                max_length= MAX_SRC_LEN, 
                                                truncation=True,
                                                padding='max_length',
                                                return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([target], 
                                                max_length=MAX_TGT_LEN,
                                                truncation=True,
                                                padding='max_length',
                                                return_tensors='pt')
        y = target['input_ids']
        target_id = y[:, :-1].contiguous()
        target_label = y[:, 1:].clone().detach()
        target_label[y[:, 1:] == self.tokenizer.pad_token_id] = -100 #in case the labels are not provided, empty string
        return source['input_ids'], source['attention_mask'], target_id, target_label

    
    def prepare_df(self, df):
        source_ids, source_masks, target_ids, target_labels = [], [], [], [] 
        for _, row in df.iterrows():
            source_id, source_mask, target_id, target_label = self.encode_text(row.text, row.target)
            source_ids.append(source_id)
            source_masks.append(source_mask)
            target_ids.append(target_id)
            target_labels.append(target_label)

        # Convert the lists into tensors
        source_ids = torch.cat(source_ids, dim=0)
        source_masks = torch.cat(source_masks, dim=0)
        target_ids = torch.cat(target_ids, dim=0)
        target_labels = torch.cat(target_labels, dim=0)
        # splitting the data to train, validation, and test
        return TensorDataset(source_ids, source_masks, target_ids, target_labels)
    
    def prepare_data(self):
        # splitting the data to train, validation, and test
        data = self.prepare_df(self.data)
        train_size, val_size = int(0.8 * len(data)), int(0.1 * len(data))
        test_size = len(data) - (train_size + val_size)
        self.train_dat, self.val_dat, self.test_dat = \
            random_split(data, [train_size, val_size, test_size])
    
    def forward(self, batch, batch_idx):
        source_ids, source_mask, target_ids, target_labels = batch[:4]
        return self.model(input_ids = source_ids, attention_mask = source_mask, 
                          decoder_input_ids=target_ids, labels=target_labels)
        
    def training_step(self, batch, batch_idx):
        loss = self(batch, batch_idx)[0]
        return {'loss': loss, 'log': {'train_loss': loss}}
    
    def predict_step(self, batch, batch_idx: int, dataloader_idx: int = 0):
        return self(batch, batch_idx)
        return " ".join(map(self.decode, y_hat))

    def validation_step(self, batch, batch_idx):
        loss = self(batch, batch_idx)[0]
        return {'loss': loss, 'val_loss': loss}

    def validation_epoch_end(self, outputs):
        loss = sum([o['loss'] for o in outputs]) / len(outputs)
        out = {'val_loss': loss}
        return {**out, 'log': out}

    def test_step(self, batch, batch_idx):
        loss = self(batch, batch_idx)[0]
        return {'loss': loss}

    def test_epoch_end(self, outputs):
        loss = sum([o['loss'] for o in outputs]) / len(outputs)
        out = {'test_loss': loss}
        return {**out, 'log': out}
    
    def train_dataloader(self):
        return DataLoader(self.train_dat, batch_size=self.batch_size,
                          num_workers=4, sampler=RandomSampler(self.train_dat))

    def val_dataloader(self):
        return DataLoader(self.val_dat, batch_size=self.args.bs, num_workers=4,
                          sampler=SequentialSampler(self.val_dat))

    def test_dataloader(self):
        return DataLoader(self.test_dat, batch_size=self.args.bs, num_workers=4,
                          sampler=SequentialSampler(self.test_dat))    

    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=self.args.lr, eps=1e-4)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=0,
            num_training_steps=self.args.max_epochs * len(self.train_dat))
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}
    
    def generate_summary(self, ctext, summ_len=150, text='', beam_search=2, repetition_penalty=2.5):
        source_id, source_mask, target_id, target_label = self.encode_text(ctext, text)
        self.model.eval()
        with torch.no_grad():
            generated_ids = self.model.generate(
                input_ids = source_id,
                attention_mask = source_mask, 
                max_length=summ_len, 
                truncation=True,
                num_beams=beam_search,
                repetition_penalty=repetition_penalty, 
                length_penalty=1.0, 
                early_stopping=True
                )
            prediction = [self.tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
        if len(text) > 0:
            target = [self.tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in target_id]
            scores = self.scorer.score(target[0], prediction[0])
            return prediction, scores
        else:
            return prediction
        

    def save_core_model(self):
        store_path = join(self.args.output, self.args.name, 'core')
        self.model.save_pretrained(store_path)
        self.tokenizer.save_pretrained(store_path)
        
    @staticmethod
    def add_model_specific_args(parent_parser):
        p = ArgumentParser(parents=[parent_parser], add_help=False)
        p.add_argument('-m', '--model', type=str, default='t5-base',
                       help='name of the model or the path pointing to it')
        p.add_argument('--bs', '--batch_size', type=int, default=2)
        p.add_argument('--source_len', type=int, default=120)
        p.add_argument('--summ_len', type=int, default=700)
        return p

In [16]:
def default_args():
    p = ArgumentParser()
    p.add_argument('-p', '--path', type=str,  
                   default='/content/gdrive/My Drive/Colab Notebooks/data/text_summarization_t5/news_summary.csv',
                  help='path to the data file')
    p.add_argument('-o', '--output', type=str, default='/tmp/tpu-template',
                  help='path to the output directory for storing the model')
    p.add_argument('-n', '--name', type=str, default='google/t5-v1_1-xxl',
                  help='this name will be used on tensorboard for the model')
    p.add_argument('-t', '--trials', type=int, default=1,
                  help='number of trials for hyperparameter search')
    p.add_argument('--seed', type=int, default=0, help='randomization seed')
    p = T5Finetuner.add_model_specific_args(p)
    p = pl.Trainer.add_argparse_args(p)
    args,_ = p.parse_known_args()
    args.max_epochs = 2
    return args

def default_args():
    p = ArgumentParser()
    args,_ = p.parse_known_args()
    args.max_epochs = 2
    #args.model = "google/flan-t5-small"
    #args.model = "google/flan-t5-xl"
    #args.model = "google/flan-t5-large"
    args.model = "google/mt5-base"
    #args.model = "google/flan-t5-base"
    args.output = f"./{args.model.replace('/','_')}"
    args.name = "DESCRIPCION_PROPIEDADES"
    args.bs = 1 # batch size
    return args

def optuna_objective(trial):
    args = default_args()
    # sampling the hyperparameters
    args.lr = trial.suggest_categorical("lr", [1e-6, 5e-6, 1e-5, 5e-5, 1e-4])
    # setting up the right callbacks
    cp_callback = pl.callbacks.ModelCheckpoint(
        join(args.output, args.name, f"trial_{trial.number}", "{epoch}"),
        monitor="val_loss", mode="min")
    pr_callback = PyTorchLightningPruningCallback(trial, monitor="val_loss")
    metrics_callback = MetricsCallback()
    summarizer = T5Finetuner(args, df_text)         # loading the model
    trainer = pl.Trainer.from_argparse_args(      # loading the trainer
        args, 
        accelerator="gpu",
        devices=1,
        default_root_dir=args.output, gradient_clip_val=1.0,
        #checkpoint_callback=cp_callback,
        callbacks=[metrics_callback],
        #early_stop_callback=pr_callback, 
        num_sanity_val_steps=-1,
        #auto_scale_batch_size="power",
        # select TensorBoad or Wandb logger
        logger=TensorBoardLogger(join(args.output, 'logs'), name=args.name, version=f'trial_{trial.number}')
        )
  
    trainer.fit(summarizer)                       # fitting the model
    trainer.test(summarizer)                      # testing the model
    return min([x['val_loss'].item() for x in metrics_callback.metrics])

In [17]:
#NEW_MODEL="/home/gvillarroel/dev/synthetic-data-for-text/notebooks/google_flan-t5-base/logs/DESCRIPCION_PROPIEDADES/trial_0/checkpoints/epoch=1-step=436696.ckpt"
NEW_MODEL = "/home/gvillarroel/dev/synthetic-data-for-text/notebooks/google_mt5-base_2/logs/DESCRIPCION_PROPIEDADES/trial_0/checkpoints/epoch=19-step=1058820.ckpt"
NEW_MODEL = "/home/gvillarroel/dev/synthetic-data-for-text/notebooks/google_mt5-base_2/logs/DESCRIPCION_PROPIEDADES/trial_0/checkpoints/epoch=19-step=1058820-v1.ckpt"
new_model = T5Finetuner.load_from_checkpoint(NEW_MODEL)



You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [31]:
from textwrap import wrap
from numpy import random

new_model.model.eval()
new_model.model.cuda()
record = df_text.sample(1).iloc[0]
def gen_new_text(record):
    #encoded = new_model.tokenizer.batch_encode_plus(record.text, max_length= MAX_SRC_LEN, truncation=False,padding=False,return_tensors='pt')
    encoded = new_model.tokenizer.batch_encode_plus(record.text, 
                                                max_length= 120, 
                                                truncation=True,
                                                padding='max_length',
                                                return_tensors='pt')
    #encoded = new_model.tokenizer("<sep>".join(record.text), return_tensors="pt")
    
    max_length= random.randint(200, 600)
    min_length = random.randint(20, max_length-5)
    min_length=60
    max_length=100
    input_ids =encoded['input_ids'].cuda() 
    att = encoded['attention_mask'].cuda()
    y_hat = new_model.model.generate(
        inputs= input_ids,
        attention_mask=att,
        num_beams=1,
        min_length=min_length,
        max_length=max_length,
        repetition_penalty=5.5,
        length_penalty=0.1,
        early_stopping=True,
        temperature=random.random(),
        use_cache=False,
        top_p=random.random(),
        #top_k=0,
        do_sample=True
    )
    del input_ids
    del att
    return " ".join([new_model.tokenizer.decode(gen_id, skip_special_tokens=True) for gen_id in y_hat

'description this data:  fecha 2022-02-23 precio 9.200 UF tipo Departamento transacción Venta región Metropolitana de Santiago comuna Las Condes dormitorios 4.0 baños 4.0 constuidos 135.0 terreno 140.0 precio_real 9200.0 titulo: Departamento en Venta en Las Condes 4 dormitorios 3 baños dirección: Parque Arauco / Parque Araucano Las Condes, Metropolitana de Santiago'

'favorecida vista despejada. 996944077, metros aproximados terrazas propiedades@gmail decom En venta hermosa casa en sector el Abrazo con Rosas solidas orientación nor oriente-orientE Propiedades El primer Edificio cuenta lavandería funcionando todo un año Parque Arauco, Las Condes. 140 m2 totales), living comedor con salida directa al balcón terraza amplia de 20m2. 4 habitaciones 3 baños principal en suite (con ventana) hall acceso 2 estacionamientos 1 bodega Gastos comuneς $280.000 aprox Contribuciones $200.000 c/cuota Edificio cuenta lavandería central panorámicas Cercano del mall Costanera Center'

'Amplio departamento en excelente ubicación, a pasos de Parque Arauco y Parque Araucano, condominio con acceso por Av. Kennedy y por Cerro Colorado. Departamento en piso 11, de 3 dormitorios más servicios completos.  Hall de entrada y pasillo de distribución. Amplio living comedor con agradable terraza. Cocina equipada y amoblada. Logia. Pieza y baño de servicios. Dos dormitorios de buen tamaño que comparten un baño completo. Dormitorio principal ampliado, en suite. Baño con ventilación natural. 2 Estacionamientos y bodega. Gastos común $280.000. Condominio cuenta con grandes y lindos parques, juegos infantiles, estacionamientos de visitas y control de acceso 24/7.'

In [33]:
#import swifter
data_g  = df_text.apply(gen_new_text, axis=1)
df_new_text = df_text.assign(generated=data_g)
df_new_text.to_parquet(f"m{df_text.shape[0]}.parquet")